In [4]:
import sys
import networkx as nx
import matplotlib as plt
import numpy as np
import re
import chess
import subprocess
import timeit
import pickle
import stockfish
from stockfish import Stockfish
import chess
import chess.engine
import pandas as pd

engine = chess.engine.SimpleEngine.popen_uci("../stockfish-10-64")
stockfish = Stockfish("../stockfish-10-64")

In [5]:
def varWeighted(scores, counts):
    if (len(counts)<2):
        return None
    else:
        weightedMean = sum([a*b for a,b in zip(scores,counts)])/sum(counts)
        scoreDiffs = [(score - weightedMean)**2 for score in scores]
        V1 = sum(counts)
        V2 = sum([count**2 for count in counts])
        var = (V1/(V1**2-V2)) * sum([a*b for a,b in zip(counts,scoreDiffs)])
        return var
    
def get_score(san):
    board = chess.Board()
    for move in san:
        board.push_san(move)
    currFen = board.fen()
    score = engine.analyse(board, chess.engine.Limit(time=.05), info=chess.engine.INFO_SCORE)
    if('#' in str(score['score'])):
        if('-' in str(score['score'])):
            return -39765
        else:
            return 39765
    else:
        score = int(str(score["score"]))
    return score

def san_to_fen(san):
    board = chess.Board()
    for move in san:
        board.push_san(move)
    currFen = board.fen()
    return currFen

def get_node_sd(node):
    scores = []
    counts = []
    try:
        for neighbor in list(g.neighbors(node)):
            scores.append(nx.get_node_attributes(g, 'score')[neighbor])
            counts.append(nx.get_node_attributes(g, 'movelistCount')[neighbor][node])
        return node, np.sqrt(varWeighted(scores,counts))
    except:
        return node, None

In [6]:
g = nx.read_gpickle('full690k.gpickle')
len(g.nodes())

10346

In [38]:
eco_df = pd.read_csv('../eco/a.tsv', sep = '\t')

In [40]:
eco_df.head()

,eco,name,fen,moves
0,A00,Amar Gambit,rn1qkbnr/ppp2ppp/8/3p4/5p2/6PB/PPPPP2P/RNBQK2R...,g1h3 d7d5 g2g3 e7e5 f2f4 c8h3 f1h3 e5f4
1,A00,Amar Opening: Gent Gambit,rn1qkbnr/ppp2ppp/8/3p4/8/6PB/PPPPP3/RNBQ1RK1 b...,g1h3 d7d5 g2g3 e7e5 f2f4 c8h3 f1h3 e5f4 e1g1 f...
2,A00,Amar Opening: Paris Gambit,rnbqkbnr/ppp2ppp/8/3pp3/5P2/6PN/PPPPP2P/RNBQKB...,g1h3 d7d5 g2g3 e7e5 f2f4
3,A00,Amar Opening,rnbqkbnr/pppppppp/8/8/8/7N/PPPPPPPP/RNBQKB1R b...,g1h3
4,A00,Amsterdam Attack,r1bqkb1r/ppp2ppp/2np1n2/4p3/2P5/1PN1P3/P2P1PPP...,e2e3 e7e5 c2c4 d7d6 b1c3 b8c6 b2b3 g8f6


In [16]:
test_list = list(g.nodes())[:2]

In [17]:
subs = 'rnbqkbnr/pppppppp/8/8/8/2N5/PPPPPPPP/R1BQKBNR b KQkq'
res = [i for i in test_list if subs in i]

In [18]:
res

['rnbqkbnr/pppppppp/8/8/8/2N5/PPPPPPPP/R1BQKBNR b KQkq - 1 1']

In [19]:
eco_list = list(eco['fen'])

In [33]:
sd_list = []
node_fen = list(g.nodes())
for eco in eco_list:
    currFen = [fen for fen in node_fen if eco in fen]
    if(len(currFen)>0):
        currFen = str(currFen[0])
    scores = []
    counts = []
    try:
        for neighbor in list(g.neighbors(currFen)):
            scores.append(nx.get_node_attributes(g, 'score')[neighbor])
            counts.append(nx.get_node_attributes(g, 'movelistCount')[neighbor][currFen])
        sd_list.append(np.sqrt(varWeighted(scores,counts)))
#         print('{} {} ||| SD :'.format(key, game), np.sqrt(varWeighted(scores,counts)), ", total Games:", sum(counts))
    except:
        sd_list.append(None)

In [34]:
sd_list[:10]

[None,
 None,
 None,
 50.548820480816694,
 None,
 None,
 31.086393379361212,
 None,
 None,
 None]

In [41]:
eco_df['sd'] = sd_list

In [44]:
eco_df.sort_values(by = ['sd'], ascending = False)

,eco,name,fen,moves,sd
168,A02,Bird Opening: From's Gambit,rnbqkbnr/pppp1ppp/8/4p3/5P2/8/PPPPP1PP/RNBQKBN...,f2f4 e7e5,267.993470
440,A40,Englund Gambit,rnbqkbnr/pppp1ppp/8/4p3/3P4/8/PPP1PPPP/RNBQKBN...,d2d4 e7e5,124.673952
519,A45,Indian Game: Omega Gambit,rnbqkb1r/pppppppp/5n2/8/3PP3/8/PPP2PPP/RNBQKBN...,d2d4 g8f6 e2e4,108.124437
228,A05,Zukertort Opening: Lemberger Gambit,rnbqkb1r/pppppppp/5n2/8/4P3/5N2/PPPP1PPP/RNBQK...,g1f3 g8f6 e2e4,97.580736
210,A04,Zukertort Opening: Ross Gambit,rnbqkbnr/pppp1ppp/8/4p3/8/5N2/PPPPPPPP/RNBQKB1...,g1f3 e7e5,96.873629
...,...,...,...,...,...
717,A96,Dutch Defense: Classical Variation,rnbq1rk1/ppp1b1pp/3ppn2/5p2/2PP4/5NP1/PP2PPBP/...,d2d4 e7e6 c2c4 f7f5 g2g3 g8f6 f1g2 f8e7 g1f3 e...,NaN
718,A97,"Dutch Defense: Classical Variation, Ilyin-Zhen...",rnb1qrk1/ppp1b1pp/3ppn2/5p2/2PP4/2N2NP1/PP2PPB...,d2d4 f7f5 g1f3 g8f6 g2g3 e7e6 f1g2 f8e7 e1g1 e...,NaN
719,A97,"Dutch Defense: Ilyin-Zhenevsky, Winter Variation",rnb1qrk1/ppp1b1pp/3ppn2/5p2/2PP4/2N2NP1/PP2PPB...,d2d4 f7f5 g1f3 g8f6 g2g3 e7e6 f1g2 f8e7 e1g1 e...,NaN
720,A98,"Dutch Defense: Classical Variation, Ilyin-Zhen...",rnb1qrk1/ppp1b1pp/3ppn2/5p2/2PP4/2N2NP1/PPQ1PP...,d2d4 f7f5 g1f3 g8f6 g2g3 e7e6 f1g2 f8e7 e1g1 e...,NaN
